# LBPTOPTool: A python implementation of an LBP-TOP based tool used for dynamic texture classification

In [ ]:
"""
Some necessary imports
"""

from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from ipywidgets import interact, widgets
import numpy as np
import cProfile
from google.colab import drive
import os
import cv2

# Mount my drive for the source code + some datasets
drive.mount("/content/drive")
os.chdir('drive/My Drive/Colab Notebooks')
from lbptop import LBPTOPTool

Mounted at /content/drive


## Introduction

LBP's (Local Binary Patterns) have been used for decades in Computer Vision for texture classification in several different applications, arguably the most popular being face detection [1,11] and recognition [1,3,8]. Distributions of these features are extracted from image data describe the prevailing patterns that characterize a texture and can be used as feature vectors, greatly reducing dimensionality and making machine learning tasks such as classification and clustering a pretty straight-forward problem of computing dissimilarities between histograms. VLBP's are a natural extension of the LBP feature extraction method to 3-dimensional video data in order to encode information about the movement of some dynamic texture, i.e a texture that changes continuously in time. More specifically, this implementation employs a variant of VLBP that uses LBP feature extraction on three independant orthogonal planes (TOP) to extract distributions of local binary patterns across different domains. The LBPTOPTool encorpoorates the methods described to transform and classify video data, and aims to provide an abstract and simple to use interface for experimentation with texture-based Computer Vision. 

## Related Work

VLBP's have been used in a wide array of computer vision tasks. Kiruba, Shiloah, and Sunil [13] used a Hexagonal shaped variant, HVLBP's, on pre-processed video data of human activities to trannsform them into histogram data, fed into an autoencoder structure and reported 85% to near 99% percent activity classification accuracy. Another interesting application of VLBP is use in facial expression recognition [7], where front facing video data of people emoting was segmented into different areas based on expression recognition relevance subsequently turned into histograms using TOP-LBP's and used in predicting the displayed sentiment. Furthermore in [5], "vanilla" VLBP's, TOP-LBP's as well as CVLB & CLBPTOP's, which are a variant of the former, normalized and thresholded against a combination of center pixel gray scale intensity combined with the mean gray scale intensities contained in each frame. Finally, there's been an interest in comparing different computational strategies for video data classification using VLBP's, contrasting multi-frame to one-shot (single frame) prediction [14].

## Background

### LBP's

Consider a grayscale image as a 2D grid (matrix) of numbers, each one corresponding to a pixel gray scale intensity. Denote this matrix as $g$. We would like to encode variation of gray scale intensities in an easy to grasp and manipulate format. This is where LBP's come in.

Suppose that we take a set of sample points from the neighborhood of a central pixel and contrast their gray scale value with that of said central pixel, this should give as a sense of the variation around this area. For this purpose, consider a circle of radius $R$ centered around some central point $c = (c_x,c_y)$ with $P$ evenly distanced sample points on the circumferance of this circle. More formaly, the LBP centered around around $c$ has the numerical value of :

$$LBP_{(R,P)}(c_x,c_y) = \sum^P_{p = 0}{2^ps(g(x_p,y_p) - g(c_x,c_y))}$$

Where $x_p = c_x + Rcos(\frac{2\pi p}{P}), y_p = c_y + Rsin(\frac{2\pi p}{P})$ and $s$ is the unit step function.

![lbp](lbp.png)
![lbpcircle](lbpcircle.png)

If we then "threshold" each one of the sample point gray scale intensities with that of the central one by subtracting this value from their own, and assigning 0 or 1 labels to those pixels that are negatively or positively valued accordingly, we can derive to binary sequence description of the local intensity variation by simply reading the labels in a clockwise manor. These sequences are what we call **Local Binary Patterns (LBP's)**.

Note the similarity that this process has to computing the derivative of a 2D function. An LBP is essentialy a numerical approximation of the derivative of the gray scale intensity of an image valued through a unit step function.

The distribution of these patterns in an image convays important identifying information about the visual content of the image, as such we usually construct histograms with counts of pattern occurances, that represent the image as feature vectors. 

These vectors can be used as model data to be compared with some novel data to be classified accordingly. More specifically, we compute the dissimilarity between two histograms using some measure such as $chi2$ (chi squared) or the log likelihood.

A quick overview of the previous dissimilarity measures: If  $0 < m_i, s_i < 1, i = 1, ..., N$ are frequencies of some pattern contained in histograms $H_M, H_S$ of the model and the sample accordingly, then the $chi2$ and log-likelihood dissimilarity measures are computed as such :


$$ chi2(H_M,H_S) = \sum^N_{i=0}\frac{(m_i - s_i)^2}{m_i + s_i}$$

$$ \mathcal{L}(H_M,H_S) = -\sum^N_{i=0}s_ilog(m_i)$$

An important thing to note about LBP's is that they have the property of being invariant to changes in luminescence, due to the relative contrast of each neighbooring sample point with the central one, i.e if we were to adjust the gray scale intensities of all the points within the radius by the same amount, the resulting LBP would still be the same.




#### Rotational Invariance

In addition to luminescence invariance we can achieve rotational invariance in LBP's by reducing all of the patterns that exist within the same group of patterns that are rotationally symmetrical to each other, meaning those that can be derived from one another via rotation. More specifically, all rotationally invariant LBP's qualified as $LBP^{ri}_{R,P}$ are represented by the minimum numerical value LBP within the previously described group, i.e:

$$LBP^{ri}_{R,P}(c_x, c_y) = \underset{i = 1,...,N}{min}\{ROR(LBP_{R,P}(c_x, c_y), i)\}$$


Where the operator $ROR(LBP_{R,P}(x,y), n)$ denotes transformation of the LBP via rotation to the right by one bit $n$ times.

This reduction of the set of LBP's to a mere subset can be useful in reducing dimensionallity since it limits the number of histogram bins.

![rotationinvariant](rotationinvariant.png)




#### Uniform LBP's 

Uniform patterns are frequently occuring patterns in real dynamic textures that have at most two circular bit transitions in their bit sequence. An in depth exploration of why this is true can be found in [15]


Non uniform patterns appear less often in image data and are thus congregated into a single category, that of the "uniform" lbp. Thus we may ignore non-uniform patterns whenever they emerge withouth much loss of information, achieving dimensionality reduction.

"Circular bit transitions" refers to the number of 1->0 or 0->1 transitions that can be found in the pattern sequence, if we were to consider it as repeating from the starting digit after its end, as if it were wrapped around a circle.

Rotationally invariant and uniform LBP's are commonly qualified as $riu$. 

### VLBP's

An LBP is nothing but the extension of the LBP to the 3D domain of video data. A VLBP operator is three LBP's put together, essentialy sampling three different consecutive moments in time in LBP fashion, by thresholding them with a center pixel gray scale value and assigning 0/1 labels to the negative/positive values that were derived. The resulting pattern is a binary sequence that emerges by putting together the assigned binary labels in the order shown by the figure bellow.

![vlbp](vlbp.png)

The three LBP's usually have a distance of $L$ frames between them, essentialy forming a cylinder volume of length $2L$, radius $R$, and a total of  a more formally the numerical value of a VLBP is computed as such:

$$ VLBP_{(L,R,P)}(x,y,z) = \sum^{3P + 2}_{i=1}2^is(g(x_i,y_i,z_i) - g(x,y,z))$$

Where the points $(x_i,y_i,z_i)$ are ordered in the way presented in the figure.

Similarly, we can instantiate rotationally invariant counterpart of the VLBP by splitting the VLBP into 5 distinct components: $p_{front}$ and $p_{back}$, the sample points that are $-L, +L$ frames apart of the central point, in the same x,y position, as well as $LBP_{-L}, LBP_0, LBP_{+L}$ the lbp's around the central point at the previous instance $-L$ frames apart, the LBP around the central point at its current instance, and the LBP around its next instance $+L$ frames in the future than the current frame. There is no need to somehow modify the points $p_{front}$ and $p_{back}$ since they dont have a specific orientation in space, whereas with the LBP's we simply have to convert them to their rotationally invariant equivalent. Similarly, we can obtain uniform VLBP's by simply limiting the set of legal LBP components to uniform instances.

The methodology for analyzing video data using VLBP's is similar to the 2D case, we simply iterate across the data volume using the VLBP operator to extract the pattern distribution of the volume in histogram form. These histograms are then used as a feature vector representation of the data.



### TOP-LBP's

The main drawback of the vanilla VLBP is the immense quantity of different patterns that come up when analyzing dynamic volumes even for a small quantity of sample points. For $P$ sample points per LBP, the volume contains $2^{3P + 2}$ sample points. TOP-LBP's counter this issue by removing corner sample points of the volume, and considering the operator as a set of Three Orhogonal Planes (TOP) that intersect each other at the center point (see figure bellow) 

![toplbp](toplbp.png)

Each orthoganal plane has its own independant LBP operator, lets call them $LBP_{xy}, LBP_{yt}, LBP_{xt}$ for each corresponding plane. For each plane we use the operator to extract the prevailing patterns and congregate their occurances in dedicated histograms. These histograms are then concatenated into a single feature vector representation of the data volume. This way, the total amount of distinct patterns i.e the feature vector dimensionality is limited to merely $3 \times 2^P$, a vast improvement compared to vanilla LBP.

![tophistograms](tophistograms.png)

TOP-LBP's are highly customizable due to the independence of their parts, each LBP can have its own radius value, number of sample points, be rotationaly invariant and or uniform, allowing for flexibility in optimizing specific tasks. For this reason, and due to promising experimental results reported in other work [2,7] i have opted for this variant of VLBP for my implementation 

## Implementation

This section outlines the implementation of the lbptop python module without going into rigorous detail or presenting the full code. For those interested, the code can be found on github. I have taken care to document the code to a degree i myself find satisfactory.

### LBTOPTool Object Interface

The LBPTOPTool provides an easy to use and simple interface that performs two main utilities: data transformation, and prediction, using an embedded KNN classifier object. 

**transform**: The transform method is the most essential part of the class. The transform method receives a 3D data volume (numpy ndarray) that contains a numerical description of the data we would like to classify and transforms it into a series of histograms describing pattern distribution in the data across three different domains (planes). This is done by iterating across every pixel and individually computing the pattern label that corresponds to the given pixel based on its neighboor's pixel intensities.

The xy, yt and xt planes (often presented in that order in this implementation) each have their own pattern distributions described by histograms that are concatenated into a common vector as described previously.

These histograms can then be used as training set feature vectors to train any classifier, or alternatively, the fit method can be used to directly transform and cache the training set as class state, using the a KNN classifier to make predictions. 

**fit / predict**: Fitting essentialy just creates a dataset of samples and their corresponding class labels, caching them as KNNClassifer object state. When using the KNNClassifier to make predictions each one of the cached histogram is compared to the sample histogram using chi2 dissimilarity or log-normal dissimilarity. The dissimilarity values of the K likeliest neighboors (where K is a model parameter) are then used to issue "votes" with an importance score inversely analogous to the dissimilarity value for each class. This is to say that for each class $c$ the total class votes value for some sample $s$ whose class we sould like to predict is computed as:

$$ class\_votes(c) = \sum^K_i \frac{in\_class(m_i,c)}{diss(m_i,s)}$$

Where $m_i, i = 1, \dots, K$, are the K-nearest neighboor's "model histograms", $diss$ is the dissimilarity between the model and sample histogram and the operator $in\_class$ is defined as:

$$
in\_class(m_i,c) = \left\{
\begin{array}{ll}
1, \text{if }m_i\text{ is in class }c\\
0, \text{otherwise} 
\end{array} 
\right. 
$$

Perhaps it is inefficient to simply iterate through the entire training set as prediction time scales linearly as datasets get larger and larger. This could be improved upon in a later implementation

The KNNClassifier class is a nested class of LBTOPTools and it is not intended to be used seperately from it.

Additionaly, when fitting the classifier one can opt to denoise & balance the dataset by removing any "Tomek links" from the dataset

### Denoising and balancing data by removing Tomek Links

"Tomek links" or T-Links are datapoints, in a set partitioned into discrete classes, that are closer to each other than with any other data point of their own class [4]. Put more formally, for some samples $x$ and $y$ in classes $C_0,C_1$, the tuple $(x,y)$ is a Tomek link if:

$$ \text{There is no } z_0 \in C_0 \text{ such that } d(x,z_0) < d(x,y) \\
   \text{ or } z_1 \in C_1 \text{ such that }   d(z_1,y) < d(x,y) $$

Where $d$ is some dissimilarity measure.

Geometrically speaking, Tomek links are points where two classes seem to interweave, making it hard to discern whether a sample belongs to one class or the other.

Removing T-Links has a twofold purpose: To denoise and to rebalance. Samples that stray far away from the mean of the class can be perceived as "noise", outliers that dont convay any useful information about the central tendency of the class and should be wiped out. Secondly, the proposed method for removing Tomek Links, states that in order to "break" these links only one sample should be removed: the one belonging to the majority class. This has the effect of slightly rebalancing the data contained in both classes.
<br>
<br>
![Visual representation of Tomek Links removal](tomek.png)

Removing T-Links has been shown to improve classification accuracy of the KNN Classifier[10], this makes sense as it increases class seperabillity and makes it easier to distinguish the class of samples that lie near the border.

Tomek link removal can be toggled on and off via the "denoise" argument of the LBPTOPTool, with the default being "False"

### Rotationaly invariant and uniform patterns

As we discussed previously, it makes sense to limit our focus to merely a subset of binary patterns that may emerge when analyzing a dynamic pattern.

In order to limit our focus to only rotationally invariant and patterns we create "lookups", i.e dictionaries that map each pattern to a corresponding rotational invariant one that is representative of its entire permutation set, including the original pattern. For each one of these sets, the binary sequence that corresponds to the minimum numerical value is selected as representative. To translate any pattern to its representative, we simply create pattern-to-representative pairings within the lookup structure, i.e dictionary entries that map patterns (keys) to rotationally invariant representatives (values). XY, YT and XT lookups are separate from one another allowing for flexibly selecting which domains to select as rotationaly invariant and or uniform.

To deal with uniform patterns, we simply count the circular bit transitions of each sequence and remove any patterns that have any more than 2 such transitions from the dictionary lookup. As a result, any time we attempt to look for a representative of pattern that is non uniform a KeyError exception is thrown. We can simply catch said exception and deal with the pattern as a special case.

The lookups are used whenever we want to increment a single histogram bin in order to account for the ocurance of a pattern when extracting LBP patterns from video data. In this case, we only increment the bins that correspond to the rotationally invariant representative and/or catch and ignore KeyErrors thrown by the lookup of non-uniform binary patterns, without noting their occurance.

One can toggle rotational invariance as well as uniformity by changing the truth value of the booleans contained in the "rotational_invariace" and "uniform" tuple arguments. Each coefficient of the tuple corresponds to each one of the XY, YT and XT plane pattern lookups, in that order.

### Data volume segmentation

 In some applications such as facial expression recognition [7], it is meaningfull to segment video data in order to capture local & temporal information contained in data. For this reason we have allowed the user to freely select the way they would like to segment the data volume, in both the spacial and temporal dimensions. In the afformentioned paper
 
The segmentation option, is a tuple of type $(n_x, n_y, n_z)$ where $n_x, n_y, n_z$ are the numbers of "slices" we would like to make of the volume, in each one of the x,y, and z directions. These segments should represent some semantic division of the volume into relevant areas of data. These segments are then analyzed individually for each sample, creating the corresponding pattern histograms and are then compared individually with the same segment histograms of other samples. The dissimilarities computed for each segment are then summed into the total dissimilarity between two data samples.

![Segmentation of facial video data](segmentation.png)

A possible way to improve class prediction accuracy would be to assign a weight to each one of these segments, discriminating against those segments which are least relevant and accentuating those that are relevant to classifying dynamic texture data. A perceptron-like model could be used here.

## Use examples and experimentation

### Some artificial examples

Consider two different sets of video data, that is to say 3-dimensional numerical data created randomly by generating two different types of visual noise. Lets call these datasets s0 and s1, each containing merely 10 samples of 124 by 124 videos with 100 frames each.

In [ ]:
s0 = np.random.normal(0, 0.5, (10, 100, 124, 124))
s1 = np.random.lognormal(0, 0.5, (10, 100, 124, 124))

As you can see in s0, the data contained is generated by drawing samples of white Gaussian noise with distribution $\mathcal{N}(0,0.5)$ similarly in s1 the data is generated by drawing random samples from a noise source with distribution $Lognormal(0,0.5)$. Interestingly enough, these categories of videos are visualy distinguishable from one another, as can be seen by observing two frames from each one of the two categories.

In [ ]:
def showframe(frame) :
    fig, axs = plt.subplots(1,2)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    axs[0].title.set_text('Set 0')
    axs[1].title.set_text('Set 1')
    axs[0].imshow(s0[0][frame])
    axs[1].imshow(s1[0][frame])

interact(showframe, frame=widgets.IntSlider(min=0, max=30, step=1, value=0))

interactive(children=(IntSlider(value=0, description='frame', max=30), Output()), _dom_classes=('widget-intera…

<function __main__.showframe>

As expected, the normally distributed samples display a symmetrical tendency around the mean resulting in a grainier texture while the lognormal is skewed, resulting in a more uniform texture with few discontinuous changes in pixel intensity (outliers).

One would expect that the embedded classifier could easilly differentiate between the two classes. Indeed this is the case, as can be seen by the following experiment: First we concatenate the samples from both categories.

In [ ]:
samples = np.concatenate((s0,s1))
labels = np.concatenate((np.zeros(len(s0)), np.ones(len(s1))))

In [ ]:
lbp = LBPTOPTool(radii=(2,2,2), sample_points=(5,5,5), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(False, False, False),
               rotation_invariant=(False, False, False), dissimilarity='chi2',
               denoise=False, interpolate = True, verbose=True)
lbp.fit(samples, labels)
test0 = np.random.normal(0, 0.5, (5, 100, 124, 124))
test1 = np.random.lognormal(0, 0.5, (5, 100, 124, 124))
test = np.concatenate((test0,test1))
test_labels = np.concatenate((np.zeros(len(test0)), np.ones(len(test1))))
predictions = lbp.predict(test)

Transforming video data to histograms...
Transforming video data to histograms...


In [ ]:
print(classification_report(predictions, test_labels))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         5
         1.0       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [ ]:
print(confusion_matrix(predictions, test_labels))

[[5 0]
 [0 5]]


We can clearly see that the LBP-TOP tool made easy work of this classification problem.

Perhaps though it is unrealistic to have a data volume where each pixel has at any instance a random grayscale intensity value, independant of its temporal and spacial neighboors... For this, lets try to run a different kind of test experiment, one that distinguishes between two oscillating surfaces vibrating at different frequencies. Also we will add a random starting phase to each oscillating surface sample.

In [ ]:
freq_high = np.zeros((10,124,124,100))
freq_low = np.zeros((10,124,124,100))

for s in range(10):
    for i,x in enumerate(np.linspace(0,2*np.pi, 124)):
        for j,y in enumerate(np.linspace(0,2*np.pi, 124)):
            for k,t in enumerate(np.linspace(0,10, 100)):
                freq_high[s, i,j,k] = np.sin(2*np.pi*t)*np.cos(12*np.pi*(y + x) + np.random.rand()*np.pi)
                freq_low[s, i,j,k] = np.sin(1*np.pi*t)*np.cos(12*np.pi*(y + x) + np.random.rand()*np.pi)

In [ ]:
def showframe(frame) :
    fig, axs = plt.subplots(1,2)
    fig.set_figheight(15)
    fig.set_figwidth(15)
    axs[0].title.set_text('Set 0')
    axs[1].title.set_text('Set 1')
    axs[0].imshow(freq_low[0,:,:,frame])
    axs[1].imshow(freq_high[0,:,:,frame])

interact(showframe, frame=widgets.IntSlider(min=0, max=99, step=1, value=0))

interactive(children=(IntSlider(value=0, description='frame', max=99), Output()), _dom_classes=('widget-intera…

<function __main__.showframe>

As can be seen above, the two dynamic textures appear identical at certain points in time, but vibrate at a different frequency, meaning that our abillity to discern between the two textures solely depends on temporal information. Lets prepare the training and test sets once again.

In [ ]:
samples = np.concatenate((freq_low,freq_high))
labels = np.concatenate((np.zeros(len(freq_low)), np.ones(len(freq_high))))

In [ ]:
lbp = LBPTOPTool(radii=(2,2,2), sample_points=(5,5,5), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(False, False, False),
               rotation_invariant=(False, False, False), dissimilarity='chi2',
               denoise=False, interpolate = True, verbose=True)
lbp.fit(samples, labels)


freq_high = np.zeros((5, 124,124,100))
freq_low = np.zeros((5, 124,124,100))
for s in range(5):
    for i,x in enumerate(np.linspace(0,2*np.pi, 124)):
        for j,y in enumerate(np.linspace(0,2*np.pi, 124)):
            for k,t in enumerate(np.linspace(0,10, 100)):
                freq_high[s, i,j,k] = np.sin(2*np.pi*t)*np.cos(12*np.pi*(y + x) + np.random.rand()*np.pi)
                freq_low[s, i,j,k] = np.sin(1*np.pi*t)*np.cos(12*np.pi*(y + x) + np.random.rand()*np.pi)
                

test = np.concatenate((freq_low,freq_high))
test_labels = np.concatenate((np.zeros(len(freq_low)), np.ones(len(freq_high))))
predictions = lbp.predict(test)

Transforming video data to histograms...
Transforming video data to histograms...


In [ ]:
print(classification_report(predictions, test_labels))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         5
         1.0       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



In [ ]:
print(confusion_matrix(predictions, test_labels))

[[5 0]
 [0 5]]


Clearly the LBP-TOP can extract the relevant temporal information to classify the oscillating surfaces without running into any issues


### DynTex database example

The DynTex database [9] is an extensive database of dynamic texture videos containing several different subjects of video data. Due to the vastness of this dataset and my limited computational resources, i have opted to limit my focus to a subset of videos where the subject is still or moving water. More specifically, the video subjects include:

- Rivers/Lakes
- Fountains
- Waterfalls
- Sinks
- Foam

<img src="river.png" alt="river" style="width: 500px;"/>
<img src="waterfall.png" alt="waterfall" style="width: 500px;"/>
<img src="sink.png" alt="sink" style="width: 500px;"/>

We prepare each inividual sample of the dataset by selecting 100 (NFRAMES) frames from each video and reshaping each frame to a 124x124 (WIDHTxHEIGHT) pixel image. To prepare a test set we repeat the same procedure on a subset of video data that has not been presented to the classifier in order to avoid showing it a different set of 100 frames that may appear in the same video and therefore have a strong visual similarity.

The video data has been uploaded to my personal google drive that i have to mount using google cloud to run the following experiments. If you wish to repeat these experiments, the dyntext dataset can be obtained from [this link](http://dyntex.univ-lr.fr/download.html), provided that you are granted permission to it.

We shall now use a variety of lbp-top objects with different attributes to predict the classes of different types of video data samples, and collect metrics on their performance.

In [ ]:
NFRAMES = 100
WIDTH = 124
HEIGHT = 124

In [ ]:
samples = []
test = []
sample_labels = []
test_labels = []

In [ ]:
def prepare_samples(data, labels, dir_name, class_name):
    
    files = os.listdir(dir_name)
    k = 0
    for filename in files:
        
        # Open video capture object on sample videos 
        cap = cv2.VideoCapture(dir_name + '/'+ filename)
        amount_of_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        
        data_sample = np.zeros((NFRAMES, WIDTH, HEIGHT))
        
        i = 0
        while amount_of_frames > 0:
            
            # Read frame
            ret, frame = cap.read()
            if ret:    
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Frame to grayscale
                amount_of_frames -= 1
                data_sample[i] = cv2.resize(gray, (WIDTH, HEIGHT))
                
                i = (i + 1) % NFRAMES
                if i == 0 :
                    data.append(data_sample)
                    labels.append(class_name)
                
            # Re-read previous frame
            else :
                pos_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)
                cap.set(cv2.CAP_PROP_POS_FRAMES, pos_frame - 1)

In [ ]:
prepare_samples(samples, sample_labels, 'fountains', 'fountains')
prepare_samples(samples, sample_labels, 'river-lake', 'river-lake')
prepare_samples(samples, sample_labels, 'waterfalls', 'waterfalls')
prepare_samples(samples, sample_labels, 'sink', 'sink')
prepare_samples(samples, sample_labels, 'foam', 'foam')
prepare_samples(test, test_labels, 'fountains_test', 'fountains')
prepare_samples(test, test_labels, 'river-lake_test', 'river-lake')
prepare_samples(test, test_labels, 'waterfalls_test', 'waterfalls')
prepare_samples(test, test_labels, 'sink_test', 'sink')
prepare_samples(test, test_labels, 'foam_test', 'foam')

#### 5 sample points, non uniform, non rotation invariant, no T-link removal

In [ ]:
lbp = LBPTOPTool(radii=(2,2,2), sample_points=(5,5,5), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(False, False, False),
               rotation_invariant=(False, False, False), dissimilarity='chi2',
               denoise=False, interpolate = True, verbose=True)
lbp.fit(np.array(samples), np.array(sample_labels))
predictions = lbp.predict(np.array(test))

Transforming video data to histograms...
Transforming video data to histograms...


In [ ]:
print(confusion_matrix(predictions, test_labels))

[[ 7  0  0  0  0]
 [ 0  9  0  0  0]
 [ 0  2 16  0  0]
 [ 0  0  0  4  0]
 [ 0  0  0  0  8]]


#### 5 sample points, non uniform, rotation invariant (xy plane),  no T-link removal

In [ ]:
lbp = LBPTOPTool(radii=(2,2,2), sample_points=(5,5,5), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(False, False, False),
               rotation_invariant=(True, False, False), dissimilarity='chi2',
               denoise=False, interpolate = True, verbose=True)
lbp.fit(np.array(samples), np.array(sample_labels))
predictions = lbp.predict(np.array(test))

Transforming video data to histograms...
Transforming video data to histograms...


In [ ]:
print(classification_report(predictions, test_labels))

              precision    recall  f1-score   support

        foam       1.00      1.00      1.00         7
   fountains       0.82      0.82      0.82        11
  river-lake       0.75      0.86      0.80        14
        sink       1.00      1.00      1.00         4
  waterfalls       1.00      0.80      0.89        10

    accuracy                           0.87        46
   macro avg       0.91      0.90      0.90        46
weighted avg       0.88      0.87      0.87        46



In [ ]:
print(confusion_matrix(predictions, test_labels))

[[ 7  0  0  0  0]
 [ 0  9  2  0  0]
 [ 0  2 12  0  0]
 [ 0  0  0  4  0]
 [ 0  0  2  0  8]]


#### 8 sample points, uniform (xy plane), rotation invariant (xy plane), T-links removed 

In [ ]:
lbp = LBPTOPTool(radii=(2,2,2), sample_points=(8,8,8), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(True, False, False),
               rotation_invariant=(True, False, False), dissimilarity='chi2',
               denoise=True, interpolate = True, verbose=True)
lbp.fit(np.array(samples), np.array(sample_labels))
predictions = lbp.predict(np.array(test))

Transforming video data to histograms...
Denoising data by removing Tomek Links...
Removed 0 samples from the dataset
Transforming video data to histograms...


In [ ]:
print(classification_report(predictions, test_labels))

              precision    recall  f1-score   support

        foam       1.00      1.00      1.00         7
   fountains       1.00      1.00      1.00        11
  river-lake       1.00      1.00      1.00        16
        sink       1.00      1.00      1.00         4
  waterfalls       1.00      1.00      1.00         8

    accuracy                           1.00        46
   macro avg       1.00      1.00      1.00        46
weighted avg       1.00      1.00      1.00        46



In [ ]:
print(confusion_matrix(predictions, test_labels))

[[ 7  0  0  0  0]
 [ 0 11  0  0  0]
 [ 0  0 16  0  0]
 [ 0  0  0  4  0]
 [ 0  0  0  0  8]]


## Profiling / Efficiency concerns

At this stage this tool exists merely as a prototype and was not intended to be a competitive implementation of the afformentioned methods. However some attempt was made to keep things running at reasonable time, accounting for the hopeless device that i developed this project on.

As such, it is meaningful to profile some of the methods of the tool and propose possible methods to reduce computation time.

First lets profile the fit method, using the virtual example we outlined in the previous section.

In [ ]:
s0 = np.random.normal(0, 0.5, (10, 100, 124, 124))
s1 = np.random.lognormal(0, 0.5, (10, 100, 124, 124))

samples = np.concatenate((s0,s1))
labels = np.concatenate((np.zeros(len(s0)), np.ones(len(s1))))

test0 = np.random.normal(0, 0.5, (5, 100, 124, 124))
test1 = np.random.lognormal(0, 0.5, (5, 100, 124, 124))
test = np.concatenate((test0,test1))

lbp = LBPTOPTool(radii=(2,2,2), sample_points=(5,5,5), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(False, False, False),
               rotation_invariant=(False, False, False), dissimilarity='chi2',
               denoise=False, interpolate = True, verbose=True)

In [ ]:
cProfile.run('lbp.fit(samples, labels)')

Transforming video data to histograms...
         15984825 function calls (15984785 primitive calls) in 91.968 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(append)
       20    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(concatenate)
       20    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(ravel)
        1    0.000    0.000   91.968   91.968 <string>:1(<module>)
       40    0.000    0.000    0.000    0.000 _asarray.py:88(asanyarray)
       20    0.000    0.000    0.000    0.000 fromnumeric.py:1689(_ravel_dispatcher)
       20    0.000    0.000    0.000    0.000 fromnumeric.py:1693(ravel)
       20    0.000    0.000    0.000    0.000 function_base.py:4636(_append_dispatcher)
       20    0.000    0.000    0.001    0.000 function_base.py:4640(append)
        3    0.000    0.000    0.000    0.000 iostr

Now lets check out the predict method

In [ ]:
cProfile.run('predictions = lbp.predict(test)')

Transforming video data to histograms...
         7998903 function calls (7998883 primitive calls) in 45.968 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(append)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(concatenate)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(ravel)
      600    0.000    0.000    0.005    0.000 <__array_function__ internals>:2(sum)
        1    0.000    0.000   45.968   45.968 <string>:1(<module>)
       20    0.000    0.000    0.000    0.000 _asarray.py:88(asanyarray)
       10    0.000    0.000    0.000    0.000 fromnumeric.py:1689(_ravel_dispatcher)
       10    0.000    0.000    0.000    0.000 fromnumeric.py:1693(ravel)
      600    0.000    0.000    0.000    0.000 fromnumeric.py:2087(_sum_dispatcher)
      600    0.001    0.000    0.004    0.000 from

In both instances, the crux of the computation time is the transform method. As we can see, an important slice of computation time is spent computing the LBP patterns for each domain. More specifically in bilinearly interpolating the gray-scale pixel raster.

One option could be to instead use the gray scale values of the nearest pixel  of the sample point (by rounding the coefficients), but in practice this just makes for bad classification in most cases. Nevertheless, we provide the option to  via the interpolate flag.

In [ ]:
lbp = LBPTOPTool(radii=(2,2,2), sample_points=(5,5,5), stride=(4,4,4),
               segmentation=(1,1,1), uniform=(False, False, False),
               rotation_invariant=(False, False, False), dissimilarity='chi2',
               denoise=False, interpolate = True, verbose=True)

In [ ]:
cProfile.run('lbp.fit(samples, labels)')

Transforming video data to histograms...
         15984825 function calls (15984785 primitive calls) in 92.013 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(append)
       20    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(concatenate)
       20    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(ravel)
        1    0.000    0.000   92.013   92.013 <string>:1(<module>)
       40    0.000    0.000    0.000    0.000 _asarray.py:88(asanyarray)
       20    0.000    0.000    0.000    0.000 fromnumeric.py:1689(_ravel_dispatcher)
       20    0.000    0.000    0.000    0.000 fromnumeric.py:1693(ravel)
       20    0.000    0.000    0.000    0.000 function_base.py:4636(_append_dispatcher)
       20    0.000    0.000    0.001    0.000 function_base.py:4640(append)
        3    0.000    0.000    0.000    0.000 iostr

In [ ]:
cProfile.run('predictions = lbp.predict(test)')

Transforming video data to histograms...
         7998903 function calls (7998883 primitive calls) in 48.436 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(append)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(concatenate)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(ravel)
      600    0.000    0.000    0.005    0.000 <__array_function__ internals>:2(sum)
        1    0.000    0.000   48.436   48.436 <string>:1(<module>)
       20    0.000    0.000    0.000    0.000 _asarray.py:88(asanyarray)
       10    0.000    0.000    0.000    0.000 fromnumeric.py:1689(_ravel_dispatcher)
       10    0.000    0.000    0.000    0.000 fromnumeric.py:1693(ravel)
      600    0.000    0.000    0.000    0.000 fromnumeric.py:2087(_sum_dispatcher)
      600    0.001    0.000    0.004    0.000 from

Some computation time could be reduced by getting rid of the costly sine/cosine computation of the coefficients of the sample points. This could be done by limiting flexibility on the amount of allowed sample points, so that precomputed templates of points may be used. Another idea that requires a bit of tinkering with the sample polygon geometry is to compute differentials of sample points in the x and y direction when moving from one sample point to the next, by simply adding this difference to the current sample point. I am reminded of a fast circle rasterization algorithm [12]...

Finally, the problem of transforming a data volume into LBP histograms can be considered "embarassingly parallel" and could benefit greatly by using GPU speedup.

## Conclusion

To summarize, there is a great deal of customizabillity to VLBP's. The methods and techniques used in a texture-based Computer Vision pipeline are very many and the combinations of said methods should be selected carefully for each application. It must be noted that the initial goal of this project was to use VLBP's in audio-visual speech recognition but testing proved that this task was not possible using VLBP's. Perhaps more experimentation should take place here, but whatever the case, the limitations of texture based Computer Vision methods are pretty obvious against applications where the specific geometry of the visual data is key. It also needs to be noted that this tool, in its prototype phase is not efficient enough to be used in real-time applications, requiring hardware support and a possible re-tooling of the pattern distribution extraction algorithm. Finally, this tool could easilly be tweaked to implement different types of VLBP's by simply changing the methods that have to do with the derivation of the pattern at a specific pixel. 


## References

[1] [Image-based Face Detection and Recognition:“State of the Art” - Faizan Ahmad, Aaima Najam, Zeeshan Ahmed](https://arxiv.org/pdf/1302.6379.pdf)

[2] [Analysis of Volume Local Binary Patterns for Video based Smoke DetectionGaohua Lin, Qixing Zhang, Gao Xu, Lan Qian, Yongming Zhang](https://www.nfpa.org/-/media/Files/News-and-Research/Resources/Research-Foundation/Symposia/2017-SUPDET/SUPDET17-Lin-et-al.ashx)

[3] [Face Recognition with Local Binary PatternsTimo Ahonen, Abdenour Hadid, and Matti Pietik ̈ainen](http://www.ee.oulu.fi/research/imag/mvg/files/pdf/pdf_494.pdf)

[4] [Classification of Imbalance Data using Tomek Link (T-Link) Combined with Random Under-sampling (RUS) as a Data Reduction Method - Elhassan AT, Aljourf M, Al-Mohanna F, Shoukri M](https://www.researchgate.net/publication/326590590_Classification_of_Imbalance_Data_using_Tomek_Link_T-Link_Combined_with_Random_Under-sampling_RUS_as_a_Data_Reduction_Method)

[5] [Analysis of Volume Local Binary Patterns for Video based Smoke Detection - Gaohua Lin, Qixing Zhang, Gao Xu, Lan Qian, Yongming Zhang](https://www.nfpa.org/-/media/Files/News-and-Research/Resources/Research-Foundation/Symposia/2017-SUPDET/SUPDET17-Lin-et-al.ashx)

[6] [Rotated Local Binary Pattern (RLBP): Rotation invariant texture descriptor - Unknown](https://pdfs.semanticscholar.org/3e70/ff6ad37af67c92433426da5b08bd64ecbe2e.pdf)

[7] [Dynamic Texture Recognition Using Local BinaryPatterns with an Application to Facial Expressions - Guoying Zhao and Matti Pietik ̈ainen](http://www.ee.oulu.fi/research/imag/mvg/files/pdf/pdf_740.pdf)

[8] [Face Recognition with Local Binary Patterns - Timo Ahonen, Abdenour Hadid, and Matti Pietik ̈ainen](http://www.ee.oulu.fi/research/imag/mvg/files/pdf/pdf_494.pdf#cite.moghaddam%3A96a)

[9] [DynTex: A comprehensive database of dynamic textures](https://www.researchgate.net/publication/222614245_DynTex_A_comprehensive_database_of_dynamic_textures)

[10] [A KNN Undersampling Approach for Data 
Balancing - 
Marcelo Beckmann, Nelson F. F. Ebecken, Beatriz S. L. Pires de Lima ](https://www.researchgate.net/publication/283672109_A_KNN_Undersampling_Approach_for_Data_Balancing)

[11] [Face Detection Based on Multi-Block LBPRepresentation - Lun Zhang, Rufeng Chu, Shiming Xiang, Shengcai Liao, Stan Z.Li](https://www.researchgate.net/publication/221383460_Face_Detection_Based_on_Multi-Block_LBP_Representation)

[12] [A Fast Bresenham Type Algorithm For Drawing Circles by John Kennedy](https://web.engr.oregonstate.edu/~sllu/bcircle.pdf)

[13] [Hexagonal Volume Local Binary Pattern (H-VLBP) with deep stacked autoencoder for Human Action Recognition](https://www.sciencedirect.com/science/article/pii/S1389041718306739?casa_token=yf82RpDLZK4AAAAA:BHF34zPCc-FucJJoqx8IUtQFukcwPHw2yq3bAV_o6hlVuG54wq7ns9kVVH8l9t37WRywIILOnXM)

[14] [Computation Strategies for Volume Local Binary Patterns applied to Action
Recognition - F. Baumann, A. Ehlers, B. Rosenhahn & Jie Liao](https://www.tnt.uni-hannover.de/papers/data/1027/avss2014_baumann.pdf)

[15][Towards Understanding the Formation of Uniform Local Binary Patterns - Olli Lahdenoja Jonne Poikonen and Mika Laiho](https://www.hindawi.com/journals/isrn/2013/429347/)



